In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor
from ipywidgets import interact
import cartopy.crs as ccrs
import netCDF4 as nc

In [ ]:
yearmonth_full_df = pd.read_csv('C:\\Users\\conor\\General_College\\UTRECHT ADS\\Thesis\\Data\\rescale_dfs\\full_yrmon.csv', index_col = 0)

In [ ]:
yearmonth_full_df1 = yearmonth_full_df.copy()
pr_df_dict = {}
importances = np.zeros(8,)

for yr in range(2000,2019):

    #fullfull######
    unfold_full = yearmonth_full_df[yearmonth_full_df['year'] != yr] #this is the data not in the fold
    fold_full = yearmonth_full_df[yearmonth_full_df['year'] == yr] #this is the fold of one year

    fold_full_y = fold_full.pop('bio_flux') #this is getting the x and y of the fold
    fold_full_yr = fold_full.pop('year')
    fold_full_month = fold_full.pop('month')

    unfold_full_y = unfold_full.pop('bio_flux') #this is getting the x and y of the unfold
    unfold_full_yr = unfold_full.pop('year')
    unfold_full_month = unfold_full.pop('month')

    #training the full model:
    regr_full_cv = RandomForestRegressor(n_estimators = 100, random_state = 1)
    regr_full_cv.fit(unfold_full, unfold_full_y) #fitting model on the unfold
    #########################
    
    #getting feature importance
    importances += regr_full_cv.feature_importances_ 
    
    #testing the full model on full data:
    preds_full_cv = regr_full_cv.predict(fold_full) #predicting on the fold 
    
    pr_df_dict[yr] = pd.DataFrame({'bio_preds': preds_full_cv , 'month': fold_full_month})
    #####################################

pred_df = pd.concat(pr_df_dict)
importances = importances/19

In [ ]:
bio_my_preds = pred_df.merge(yearmonth_full_df, left_on = ['Unnamed: 0', 'month'], right_on = ['year', 'month'])[['year', 'month', 'bio_flux', 'bio_preds']]


In [ ]:
#predicting w/ proper 
yr_addendum_list = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18']
jung_preds_l = []

for yr in yr_addendum_list:
    jung_s_ = pd.read_csv('non0_jun_s_' + yr + '.csv')
    jung_preds_l.append(jung_s_)

jung_s_ = pd.concat(jung_preds_l).reset_index(drop = True)

In [ ]:
combo_predictions =  bio_my_preds.merge(jung_s_, 
                               left_on = ['year', 'month'], right_on = ['year', 'month'])[['year', 'month', 'bio_flux', 'bio_preds', 'flux_predictions']]

In [ ]:
#plotting

fig, ax = plt.subplots()

combo_predictions_copy.plot(x = 'yearmon', y = ['bio_flux', 'bio_preds', 'flux_predictions'], 
                            figsize = (12,6), xlabel = 'Year', ylabel = r'Net Ecosystem Exchange [$molm^{-2}s^{-1}$]',
                            ax = ax)
ax.legend(["NEE", "Combo model predictions", 'Jung style predictions'],loc='upper center', 
                 bbox_to_anchor=(0.5, -0.1),fancybox=False, shadow=False, ncol=3) 

# fig.savefig('C:\\Users\\conor\\General_College\\UTRECHT ADS\\Thesis\\Figures\\two_models_predictions_non0.jpg',
#             dpi=300, bbox_inches='tight')

In [ ]:
year_labels = list(combo_predictions['year'].unique())
act_NEE = []
combo_NEE = []
jung_NEE = []


for yr in combo_predictions['year'].unique():
    yr_df = combo_predictions[combo_predictions['year'] == yr]
    
    act_NEE.append(np.sum(yr_df['bio_flux']))
    combo_NEE.append(np.sum(yr_df['bio_preds']))
    jung_NEE.append(np.sum(yr_df['flux_predictions']))


act_NEE.append(np.mean(act_NEE))
combo_NEE.append(np.mean(combo_NEE))
jung_NEE.append(np.mean(jung_NEE))
year_labels.append('Average')

In [ ]:
x = np.arange(len(year_labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
fig.set_size_inches(12, 6)

rects1 = ax.bar(x - width, act_NEE, width, label='Actual')
rects2 = ax.bar(x , combo_NEE, width, label='Combo Model')
rects3 = ax.bar(x + width, jung_NEE, width, label='Jung style Model')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel(r'NEE $molm^{-2}s^{-1}$')
# ax.set_title('Carbon Uptake predicted for both models vs. actual results')
ax.set_xticks(x, year_labels)
ax.legend(loc='upper center',bbox_to_anchor=(0.5, -0.05),fancybox=False, shadow=False, ncol = 3)

# ax.bar_label(rects1, padding=3)
# ax.bar_label(rects2, padding=3)
# ax.bar_label(rects3, padding=3)

fig.tight_layout()
plt.gca().invert_yaxis()

# fig.savefig('C:\\Users\\conor\\General_College\\UTRECHT ADS\\Thesis\\Figures\\two_models_uptake per year.jpg',
#             dpi=300, bbox_inches='tight')

plt.show()